In [1]:
import polars as pl

import numpy as np
import json
import git
import os
# Get absolute file path
git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

from google import genai
from google.genai.types import HarmBlockThreshold, HarmCategory, SafetySetting, GenerateContentConfig


from tqdm.notebook import tqdm

import jsonlines

In [2]:
# Params
PRINT = False

write_path = git_root + "/" + "data/generated/information_extraction.jsonl"


In [3]:
# Pull in Sutta text

path_to_data = "data/web/thanissaro_scraped.jsonl"
path_to_data = git_root + "/" + path_to_data

data = []

with jsonlines.open(path_to_data) as reader:
    for obj in reader:
        data.append({"text": obj["sutta_text"],
                     "sutta":obj["sutta"]} )

df = pl.from_dicts(data)
df.head()

text,sutta
str,str
"""“Monks, these three persons ar…","""AN 3:116"""
"""Once the Blessed One was livin…","""AN 7:58"""
"""This was said by the Blessed O…","""Iti 48"""
"""On one occasion the Blessed On…","""AN 3:126"""
"""“Monks, without abandoning six…","""AN 6:73"""


In [5]:
# Provide API
API_PATH = "/home/graham/.config/my_api_keys"
with open(API_PATH, 'r') as f:
    keys = f.read()
gemini_api = json.loads(keys)['google_gemini']
client = genai.Client(api_key=gemini_api)

In [6]:
safety_settings = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
]

In [12]:
bad_extensive = []
SYSTEM_INSTRUCTION = """You are an expert data extractor specializing in Buddhist philosophy and the Pali Canon. Your primary function is to analyze a Sutta text and extract a knowledge graph based **exclusively** on the information contained within that text. You will identify the internal ontology—the entities, processes, relationships, and concepts—as they are defined and related *within the Sutta itself*.

Your output will be used to generate a knowledge graph for Graph RAG, so precision, adherence to the text, and correct formatting are paramount.

**Core Rules:**

1.  **Text-Only Extraction:** All extracted information **must** be directly present in or clearly implied by the provided Sutta text. Do not introduce any external Buddhist knowledge, interpretations, or definitions.
2.  **Internal Ontology:** Focus on how the Sutta defines its own terms and relationships. Pay close attention to metaphors, where one concept is defined in relation to another.
3.  **Graph-Ready Relationships:** Every relationship must be a "triple" with a single, specific source and a single, specific target.
4.  **Relevance Filter:** Extract only entities, processes, and relationships that are thematically significant to the Sutta's core message and its practical teachings. Ignore incidental details (e.g., "a monk sat down").

---

**ANALYSIS INSTRUCTIONS**

For the Sutta text provided below, perform the following extraction steps and format the output as a single Python dictionary.

**1. Identify Entities**
Extract significant nouns or concepts that function as "things" in the Sutta. An entity can be a being, an abstract concept treated as an object, a mental state, or a place.

*   **`name`**: The name of the entity.
*   **`type`**: [+] A broad category. Use the following **non-exhaustive list as a guide** to inform your judgment: 'Person', 'Group', 'Mental State', 'Concept as Object', 'Location', 'Physical Object', 'Figurative Language'.
*   **`description`**: A summary of the entity based *only* on its description in this text. If not described, state "Not described in this text."
*   **`goal`**: Any stated goal of the entity.
*   **`intention`**: Any stated intention of the entity.
*   **`practice_link`**: How this entity relates to a Buddhist practice, if mentioned.

**2. Identify Processes**
Extract significant actions, events, or transformations. A process describes something happening over time and typically involves one or more entities.

*   **`name`**: A concise name for the process (e.g., 'The Arising of Suffering', 'Cultivating the Path').
*   **`description`**: A high-level explanation of the process as described in the text.
*   **`impacted_entities`**: A list of entity names impacted by this process.
*   **`steps`**: Deconstruct the process into a sequence of events.
    *   **`step_description`**: Description of the step.
    *   **`starting_state`**: The state or entity before the action.
    *   **`action`**: The action being performed.
    *   **`resulting_state`**: The state or entity after the action.
*   **`goal`**: The stated or implied goal of the process.
*   **`practice_link`**: How this process relates to a Buddhist practice, if mentioned.

**3. Identify Relationships**
Extract directed connections between entities and/or processes. These are the edges of your knowledge graph.

[+] **Guiding Principles for Relationship Labels:** Before extracting, understand these principles for creating the `relationship_label`.
*   **Be Active & Simple:** Formulate the label as a concise, active verb phrase, standardized to its base form (e.g., use `LEADS_TO` instead of "leads to" or "led to").
*   **Capture the Nuance:** The label should be as specific as the text allows. Instead of a generic `RELATES_TO`, prefer `IS_A_CAUSE_OF`, `IS_A_COMPONENT_OF`, `IS_COMPARED_TO`, or `IS_AN_ANTIDOTE_FOR`.
*   **Common Relation Types to Look For (Non-Exhaustive List):**
    *   **Causal:** `CAUSES`, `LEADS_TO`, `IS_A_CONDITION_FOR`, `RESULTS_FROM`
    *   **Compositional:** `IS_PART_OF`, `CONSISTS_OF`, `IS_AN_EXAMPLE_OF`
    *   **Social/Narrative:** `INTERACTS_WITH`, `ASKS_QUESTION_OF`, `TEACHES`, `RESIDES_IN`
    *   **Comparative:** `IS_COMPARED_TO`, `IS_SUPERIOR_TO`, `IS_OPPOSITE_OF`
    *   **Definitional:** `IS_DEFINED_AS`, `IS_A_TYPE_OF`

Now, extract the relationships with the following attributes:
*   **`source`**: The name of the source Entity or Process.
*   **`target`**: The name of the target Entity or Process.
*   **`relationship_label`**: A short, general, verb-based label for the relationship, following the principles above (e.g., 'LEADS_TO', 'IS_A', 'CAUSES'). This will be the predicate in your graph triple.
*   **`specific_description`**: A detailed sentence describing how the text *specifically* frames this relationship. Capture the unique context and nuance.
*   **`medium`**: The entity or process that enables or facilitates the relationship, if specified.
*   **`outcome`**: The result of the relationship, if different from the target.

**4. Identify Concepts**
Extract key doctrinal or philosophical ideas being explained. A concept is a teaching or a principle, whereas an entity is treated as a "thing" within the Sutta's narrative.

*   **`name`**: The name of the concept (e.g., 'The Four Noble Truths', 'Impermanence').
*   **`description`**: An explanation of the concept based *only* on how it is presented in this Sutta.
*   **`practice_link`**: How this concept is connected to a specific practice.
*   **`goal`**: The goal or purpose associated with understanding this concept.

---

**OUTPUT FORMAT**

Provide your analysis in a single Python dictionary. Do not include any explanatory text before or after the dictionary. If a field has no information, use `None` for single values or an empty list `[]` for lists. Do not use a ```python delimiter.

{
  "entities": [
    {
      "name": "...",
      "type": "...",
      "description": "...",
      "goal": "...",
      "intention": "...",
      "practice_link": "..."
    }
  ],
  "processes": [
    {
      "name": "...",
      "description": "...",
      "impacted_entities": ["..."],
      "steps": [
        {
          "step_description": "...",
          "starting_state": "...",
          "action": "...",
          "resulting_state": "..."
        }
      ],
      "goal": "...",
      "practice_link": "..."
    }
  ],
  "relationships": [
    {
      "source": "...",
      "target": "...",
      "relationship_label": "...",
      "specific_description": "...",
      "medium": "...",
      "outcome": "..."
    }
  ],
  "concepts": [
    {
      "name": "...",
      "description": "...",
      "practice_link": "...",
      "goal": "..."
    }
  ]
}"""

In [ ]:
# # Load existing data
# existing_data = pl.read_ndjson(write_path)
# existing_data = existing_data.filter(
#     (pl.col("model_name").eq(model_extensive.model_name) & pl.col("prompt_type").eq("extensive"))
# )
# data_extensive = pl.from_dicts(data).filter(
#     pl.col("sutta").is_in(existing_data["sutta"]).not_()
# )
# print(f"Size of data to perform inference on: {data_extensive.height}")
# data_extensive.head()


Size of data to perform inference on: 496


text,sutta,model_output,model_name,prompt_type
str,str,str,str,str
"""Then Ven. Sāriputta went to t…","""SN 55:5""",null,null,null
"""I have heard that on one occas…","""AN 10:165""",null,null,null
"""An example of how pointed the …","""AN 5:191""",null,null,null
"""I have heard that on one occas…","""SN 4:8""",null,null,null
"""I have heard that on one occas…","""Ud 3:1""",null,null,null


In [13]:
model_id = 'gemini-2.5-flash-preview-04-17'

config = GenerateContentConfig(
      temperature=0.95,
      safety_settings=safety_settings,
      system_instruction=SYSTEM_INSTRUCTION
)

In [ ]:
bad = []
data = df.to_dicts()
for el_ in tqdm(data):
    try:
        response = client.models.generate_content(
            model=model_id,
            config=config,
            contents=el_['text']

        ).text

        el_.update(
            {
                "model_output": response,
                "model_id": model_id
            }
        )

        if PRINT == False:
            with jsonlines.open(write_path, mode='a') as writer:
                writer.write(el_) 
    except Exception as e:
        print(f"""{el_["sutta"]}: {e}""")
        bad_extensive.append(el_)


  0%|          | 0/2815 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [21]:
SYSTEM_INSTRUCTION_SIMPLE = """
You are an expert in Buddhist philosophy and the Pali Canon. Your task is to analyze the provided Sutta text and extract its key elements, focusing on the ontology of entities, processes, and relationships described within the text itself. Provide the information in a structured format. You are helping me to generate a knowledge graph for Graph RAG and document interaction for the Pali Canon. The Pali Canon is a set of suttas that are related to Buddhism. Importantly, the Buddha and his monks often spoke in metaphor. This means that entities in the Canon exist that are defined relative to other entities. There is effectively an ontology, a set of entities and processes as well as the relationships among them, that is defined within the Pali Canon itself. It is your job to help identify these entities, processes, and relationships. You should focus on extracting relationships that exist in the current Sutta. Do not include any external information in the extraction. You should focus on only extracting graph-ready information in which each relationship should have only a single source and target.

For each sutta that I provide you with follow these instructions:

**Instructions:**
1.  **Read the Sutta:** Carefully read the Sutta text provided below.
2.  **Extract Entities:** Identify significant entities (things, concepts, processes, beings, etc.) in the Sutta. For each, give its name and a description based only on what the Sutta itself says.
4.  **Extract Relationships:** Identify key and important relationships between entities and processes in the Sutta. For each, give a relationship name, a description from the Sutta, the source, the target, and a relationship type (e.g., causal, associative, constituent, etc.).
5.  **Extract Concepts:** Identify key Buddhist concepts. For each, give its name, a description based on the Sutta.
6.  **Structured Output:** Return the extracted information as a Python dictionary with the following structure:

 {
      "entities": [
        { 
        "name": "[Entity Name]", 
        "type": "[Entity Type]", 
        "description": "[Entity Description]" 
        },
         ...
      ],
      "relationships": [
        { 
        "name": "[Relationship Name]", 
        "specific_description": "[Relationship Description]", 
        "general_type": "[Relationship Type]", 
        "source": "[Source]", 
        "target": "[Target]" 
        },
         ...
      ],
      "concepts": [
        { 
        "name": "[Concept Name]", 
        "description": "[Concept Description]", 
        "type": "[verbatim/summary/inference]"
        },
         ...
      ]
    }
"""

model_simple = genai.GenerativeModel(
    "gemini-2.0-flash-exp",
    system_instruction=SYSTEM_INSTRUCTION_SIMPLE
    )


In [22]:
# Load existing data
data = []

with jsonlines.open(path_to_data) as reader:
    for obj in reader:
        data.append({"text": obj["sutta_text"],
                     "sutta":obj["sutta"]} )
        
existing_data = pl.read_ndjson(write_path)
existing_data = existing_data.filter(
    (pl.col("model_name").eq(model_simple.model_name) & pl.col("prompt_type").eq("simple"))
)
data_simple = pl.from_dicts(data).filter(
    pl.col("sutta").is_in(existing_data["sutta"]).not_()
)
print(f"Size of data to perform inference on: {data_simple.height}")
data_simple.head()


Size of data to perform inference on: 1395


text,sutta
str,str
"""“Monks, these three persons ar…","""AN 3:116"""
"""Once the Blessed One was livin…","""AN 7:58"""
"""This was said by the Blessed O…","""Iti 48"""
"""On one occasion the Blessed On…","""AN 3:126"""
"""“Monks, without abandoning six…","""AN 6:73"""


In [23]:
bad_simple = []
data_simple = data_simple.to_dicts()
for el_ in tqdm(data_simple):
    try:
        el_["model_output"] = model_simple.generate_content(
            el_["text"]
        ).text
        el_["model_name"] = model_simple.model_name
        el_["prompt_type"] = "simple"

        if PRINT == False:
            with jsonlines.open(write_path, mode='a') as writer:
                writer.write(el_) 
    except Exception as e:
        print(f"""{el_["sutta"]}: {e}""")
        bad_simple.append(el_)


  0%|          | 0/1395 [00:00<?, ?it/s]

SN 35:82: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 4. Meaning that the model was reciting from copyrighted material.


In [31]:
SYSTEM_INSTRUCTION_GENERAL = """
**Instructions:**
You are an expert in analyzing documents and extracting knowledge. Your task is to analyze the following document and identify its key elements, focusing on information explicitly stated within the document itself. Do not use prior knowledge or information not contained in the text.

The dictionary should have the following keys:
* `"main_topics"`: A list of strings representing the main topics or themes of the document.
* `"key_entities"`: A dictionary where each key is a concept or instance name and its value is either "concept" or "instance."
* `"local_relationships"`: A list of dictionaries, where each dictionary represents a relationship with keys: `"entity1"`, `"relation_phrase"`, `"entity2"` and an optional key `"relation_type"` if the relationship type is explicit in the text.
* `"descriptive_phrases"`: A dictionary where each key is an entity and its value is a list of strings representing its descriptive phrases.
* `"local_definitions"`: A dictionary where each key is an entity and its value is the string representing its local definition(s), including direct quotes from the document if available.
* `"key_phrases"`: A list of key phrases and recurring terminology from the document.

**Example Output:**
Here's an example of what the output should look like:

**Instructions:**
For this document, please identify and list the following, formatted as a Python dictionary. Follow the schema provided below strictly and only output valid JSON which is a python dictionary.

The dictionary should have the following keys:
* `"main_topics"`: A list of strings representing the main topics or themes of the document.
* `"key_entities"`: A dictionary where each key is a concept or instance name and its value is either "concept" or "instance."
* `"local_relationships"`: A list of dictionaries, where each dictionary represents a relationship with keys: `"entity1"`, `"relation_phrase"`, `"entity2"` and an optional key `"relation_type"` if the relationship type is explicit in the text.
* `"descriptive_phrases"`: A dictionary where each key is an entity and its value is a list of strings representing its descriptive phrases.
* `"local_definitions"`: A dictionary where each key is an entity and its value is the string representing its local definition(s), including direct quotes from the document if available.
* `"key_phrases"`: A list of key phrases and recurring terminology from the document.

**Example Output:**

Document: "There, monks, a monk dwells contemplating the body, ardent, clearly knowing, mindful, having put away covetousness and grief in the world. He dwells contemplating feelings, ardent, clearly knowing, mindful, having put away covetousness and grief in the world. He dwells contemplating mind, ardent, clearly knowing, mindful, having put away covetousness and grief in the world. He dwells contemplating mental objects, ardent, clearly knowing, mindful, having put away covetousness and grief in the world."

Answer: 

{
    "main_topics": ["mindfulness", "contemplation", "meditation"],
    "key_entities": {
        "monk": "concept",
        "body": "concept",
        "feelings": "concept",
        "mind": "concept",
        "mental objects": "concept",
        "covetousness": "concept",
        "grief": "concept",
        "world": "concept"
    },
    "local_relationships": [
        {
            "entity1": "monk",
            "relation_phrase": "dwells contemplating",
            "entity2": "body",
            "relation_type": "contemplation"
        },
        {
            "entity1": "monk",
             "relation_phrase": "dwells contemplating",
             "entity2": "feelings",
            "relation_type": "contemplation"
        },
      {
             "entity1": "monk",
              "relation_phrase": "dwells contemplating",
             "entity2": "mind",
            "relation_type": "contemplation"
        },
       {
             "entity1": "monk",
             "relation_phrase": "dwells contemplating",
             "entity2": "mental objects",
            "relation_type": "contemplation"
        },
        {
            "entity1": "monk",
            "relation_phrase": "having put away",
             "entity2": "covetousness",
             "relation_type": "put away"
        },
        {
            "entity1": "monk",
             "relation_phrase": "having put away",
            "entity2": "grief",
            "relation_type": "put away"
        },
         {
            "entity1": "covetousness",
            "relation_phrase": "in",
            "entity2": "world"
         },
          {
            "entity1": "grief",
             "relation_phrase": "in",
             "entity2": "world"
        }
    ],
     "descriptive_phrases": {
        "monk":["ardent", "clearly knowing", "mindful"],
        "body": ["body"],
        "feelings": ["feelings"],
        "mind":["mind"],
        "mental objects": ["mental objects"]
      },
    "local_definitions": {},
    "key_phrases": ["dwells contemplating", "having put away", "covetousness and grief"]
}
"""

model_general = genai.GenerativeModel(
    "gemini-2.0-flash-exp",
    system_instruction=SYSTEM_INSTRUCTION_GENERAL
    )


In [33]:
# Load existing data
data = []

with jsonlines.open(path_to_data) as reader:
    for obj in reader:
        data.append({"text": obj["sutta_text"],
                     "sutta":obj["sutta"]} )
        
existing_data = pl.read_ndjson(write_path)
existing_data = existing_data.filter(
    (pl.col("model_name").eq(model_general.model_name) & pl.col("prompt_type").eq("general"))
)
data_general = pl.from_dicts(data).filter(
    pl.col("sutta").is_in(existing_data["sutta"]).not_()
)
print(f"Size of data to perform inference on: {data_general.height}")
data_general.head()

Size of data to perform inference on: 1395


text,sutta
str,str
"""“Monks, these three persons ar…","""AN 3:116"""
"""Once the Blessed One was livin…","""AN 7:58"""
"""This was said by the Blessed O…","""Iti 48"""
"""On one occasion the Blessed On…","""AN 3:126"""
"""“Monks, without abandoning six…","""AN 6:73"""


In [34]:
bad_general = []
data_general = data_general.to_dicts()
for el_ in tqdm(data_general):
    try:
        el_["model_output"] = model_general.generate_content(
            el_["text"]
        ).text
        el_["model_name"] = model_general.model_name
        el_["prompt_type"] = "general"

        if PRINT == False:
            with jsonlines.open(write_path, mode='a') as writer:
                writer.write(el_) 
    except Exception as e:
        print(f"""{el_["sutta"]}: {e}""")
        bad_general.append(el_)


  0%|          | 0/1395 [00:00<?, ?it/s]

In [35]:
SYSTEM_INSTRUCTION_THINKING = """
I need to extract a complete ontology from the following document, which is a sutta from the Buddhist Pali Canon. This ontology is defined internally among all of the documents that you are going to see but you are only going to see each one at a time. This means you will not get to see all documents simultaneously to construct the ontology. 

I think an ontology consists of two main elements. First, entities, which may be objects, concepts, or processes, and second, relationships between those entities. This means I think of the ontology as a sort of graph of nodes (entities) and connections between nodes (relationships). I call this an ontology because "meaning" of something in such a system can be explained in terms of other entities and relationships. This is how things are often explained in the Pali Canon, so it seems to fit. But if you think I am missing some element, please fill it in. 

I want to do this because I want to understand that ontology. I think Buddhism is a religion that seeks to do something particularly unique and interesting in terms of human phenomenology, psychology, and experience. It however often does this through metaphor. I am hoping that identifying this ontology will help me to understand the meaning of those metaphors in terms of my own life and general human experience. This is my ultimate goal.

Your task is to generate a python dictionary that will help me to identify that ontology. I ultimately want to use the data in the dictionaries you generate to construct a graph structure that maps this ontology out and perhaps to use with RAG (retrieval augmented generation). 

Please do your best and make the decisions that make the most sense to you for helping me in my ultimate goal of understanding Buddhist ontology.
"""

model_thinking = genai.GenerativeModel(
    "gemini-2.0-flash-thinking-exp",
    system_instruction=SYSTEM_INSTRUCTION_THINKING
    )

In [38]:
# Load existing data
data = []

with jsonlines.open(path_to_data) as reader:
    for obj in reader:
        data.append({"text": obj["sutta_text"],
                     "sutta":obj["sutta"]} )
        
existing_data = pl.read_ndjson(write_path)
existing_data = existing_data.filter(
    (pl.col("model_name").eq(model_thinking.model_name) & pl.col("prompt_type").eq("thinking"))
)
data_thinking = pl.from_dicts(data).filter(
    pl.col("sutta").is_in(existing_data["sutta"]).not_()
)
print(f"Size of data to perform inference on: {data_thinking.height}")
data_thinking.head()

Size of data to perform inference on: 198


text,sutta
str,str
"""I have heard that on one occas…","""DN 16"""
"""Four times, five, I ran amok f…","""Thig 6:8"""
"""Then Ven. Anuruddha went to Ve…","""AN 3:131"""
"""I have heard that on one occas…","""SN 48:53"""
"""I have heard that at one time …","""AN 3:137"""


In [42]:
print(existing_data.head()["model_output"][0])

Here's a breakdown of the thinking process to arrive at the ontology for the given sutta:

1. **Understand the Goal:** The user wants to extract an ontology (entities and relationships) from a single Buddhist sutta. The ultimate aim is to understand Buddhist concepts related to human experience and psychology. The output should be a Python dictionary.

2. **Identify Core Entities:** The sutta explicitly mentions three "persons" as the central entities. These categories of persons form the primary structure. So, the initial entities are:
    * Person (general category)
    * Easy to Measure Person
    * Hard to Measure Person
    * Immeasurable Person

3. **Analyze Each Entity Type:** Go through the descriptions of each person type and identify their defining characteristics. These characteristics will become attributes of the entities.

    * **Easy to Measure Person:**  The description focuses on negative qualities of mind and behavior. These translate into attributes:
        * High-

In [37]:
bad_thinking = []
data_thinking = data_thinking.to_dicts()
for el_ in tqdm(data_thinking):
    try:
        el_["model_output"] = model_thinking.generate_content(
            el_["text"]
        ).text
        el_["model_name"] = model_thinking.model_name
        el_["prompt_type"] = "thinking"

        if PRINT == False:
            with jsonlines.open(write_path, mode='a') as writer:
                writer.write(el_) 
    except Exception as e:
        print(f"""{el_["sutta"]}: {e}""")
        bad_thinking.append(el_)


  0%|          | 0/1395 [00:00<?, ?it/s]

DN 16: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
Thig 6:8: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 4. Meaning that the model was reciting from copyrighted material.
AN 3:131: 429 Resource has been exhausted (e.g. check quota).
SN 48:53: 429 Resource has been exhausted (e.g. check quota).
AN 3:137: 429 Resource has been exhausted (e.g. check quota).
Khp 5: 429 Resource has been exhausted (e.g. check quota).
SN 41:10: 429 Resource has been exhausted (e.g. check quota).
AN 3:68: 429 Resource has been exhausted (e.g. check quota).
SN 27:3: 429 Resource has been exhausted (e.g. check quota).
Khp 2: 429 Resource has been exhausted (e.g. check quota).
SN 22:86: 429 Resource has been exhausted (e.g. check quota).
AN 5:148: 429 Resource ha